In [1]:
import intake
import satstac
import yaml
import inspect
import intake_xarray
import intake_stac
intake_stac.__version__

'0.3.0.post58'

# Load an Intake Catalog, Grab a Single Asset

In [2]:
url = 'https://raw.githubusercontent.com/radiantearth/stac-spec/master/examples/catalog.json'
stac_cat = intake.open_stac_catalog(url)

l1 = stac_cat
l2 = stac_cat['extensions-collection']
l3 = stac_cat['extensions-collection']['proj-example']

print(list(l1))
print(list(l2))
print(list(l3))

['extensions-collection', 'sentinel-2', 'CS3-20160503_132131_08']
['proj-example']
['B1', 'B8']


In [3]:
b1 = stac_cat['extensions-collection']['proj-example']['B1']
b8 = stac_cat['extensions-collection']['proj-example']['B8']

#print(type(b1))
#print(b1)

# Serialize as a .YAML file

[Intake _yaml code](https://github.com/intake/intake/blob/471488801bfa338dbfd91f9851b27580f4f3c3bb/intake/source/base.py#L242-L265)

``` python
def _yaml(self):
    import inspect
    kwargs = self._captured_init_kwargs.copy()
    meta = kwargs.pop('metadata', self.metadata) or {}
    kwargs.update(dict(zip(inspect.signature(self.__init__).parameters,
                       self._captured_init_args)))
    data = {
        'sources':
            {self.name: {
                'driver': self.classname,
                'description': self.description or "",
                'metadata': meta,
                'args': kwargs
            }}}
    return data

def yaml(self):
    """Return YAML representation of this data-source
    The output may be roughly appropriate for inclusion in a YAML
    catalog. This is a best-effort implementation
    """
    data = self._yaml()
    return yaml.dump(data, default_flow_style=False)
```

In [ ]:
b1data = b1._yaml() # For datasources the existing yaml implimentation works.
b1data['metadata'] = {'version':1}

## Confirm that I can save and load this yaml
with open('b1test.yaml', 'w') as f:
    f.write(yaml.dump(b1data))

b1cat = intake.open_catalog('b1test.yaml')
#list(b1cat)
#b1cat['stac_asset'] #Works!

In [ ]:
b8data = b8._yaml() # For datasources the existing yaml implimentation works.
b8data['metadata'] = {'version':1}

## Confirm that I can save and load this yaml
with open('b8test.yaml', 'w') as f:
    f.write(yaml.dump(b8data))

b8cat = intake.open_catalog('b8test.yaml')
#list(b8cat)
#b8cat['stac_asset'] #Works!

In [51]:
## Load the manually edited file to include both

l4mancat = intake.open_catalog('l4mantest.yaml')

ValidationError: Catalog 'l4mantest.yaml' has validation errors:

("missing required key 'driver'", {'stac_asset': {'args': {'chunks': {}, 'urlpath': 'https://landsat-pds.s3.amazonaws.com/c1/L8/107/018/LC08_L1TP_107018_20181001_20181001_01_RT/LC08_L1TP_107018_20181001_20181001_01_RT_B1.TIF'}, 'description': 'Band 1 (coastal)', 'driver': 'intake_xarray.raster.RasterIOSource', 'metadata': {'catalog_dir': '', 'eo:bands': [{'center_wavelength': 0.44, 'common_name': 'coastal', 'full_width_half_max': 0.02, 'name': 'B1'}], 'href': 'https://landsat-pds.s3.amazonaws.com/c1/L8/107/018/LC08_L1TP_107018_20181001_20181001_01_RT/LC08_L1TP_107018_20181001_20181001_01_RT_B1.TIF', 'plots': {'geotiff': {'cmap': 'viridis', 'data_aspect': 1, 'dynamic': True, 'frame_width': 500, 'kind': 'image', 'rasterize': True, 'x': 'x', 'y': 'y'}}, 'title': 'Band 1 (coastal)', 'type': 'image/tiff; application=geotiff'}}})
("missing required key 'driver'", {'stac_asset': {'args': {'chunks': {}, 'urlpath': 'https://lands

In [45]:
# Try to get both B1 and B8 in same YAML
b8data = b8._yaml()
l4data = l3data.copy()
l4data['stac_asset'] = b8data

#l4data = {'metadata':{'version':1}, 'sources':{}}
#for key, source in l3.items():
#    l4data['sources'][key] = source._yaml()['sources']

with open('l4test.yaml', 'w') as f:
    f.write(yaml.dump(l4data))

l4catalog = intake.open_catalog('l4test.yaml') #Validation Error can't find `driver`

ValidationError: Catalog 'l4test.yaml' has validation errors:

("missing key 'sources'", {'args': {'name': 'proj-example', 'stac_obj': {'assets': {'B1': {'stac_asset': {'args': {'chunks': {}, 'urlpath': 'https://landsat-pds.s3.amazonaws.com/c1/L8/107/018/LC08_L1TP_107018_20181001_20181001_01_RT/LC08_L1TP_107018_20181001_20181001_01_RT_B1.TIF'}, 'description': 'Band 1 (coastal)', 'driver': 'intake_xarray.raster.RasterIOSource', 'metadata': {'catalog_dir': '', 'eo:bands': [{'center_wavelength': 0.44, 'common_name': 'coastal', 'full_width_half_max': 0.02, 'name': 'B1'}], 'href': 'https://landsat-pds.s3.amazonaws.com/c1/L8/107/018/LC08_L1TP_107018_20181001_20181001_01_RT/LC08_L1TP_107018_20181001_20181001_01_RT_B1.TIF', 'plots': {'geotiff': {'cmap': 'viridis', 'data_aspect': 1, 'dynamic': True, 'frame_width': 500, 'kind': 'image', 'rasterize': True, 'x': 'x', 'y': 'y'}}, 'title': 'Band 1 (coastal)', 'type': 'image/tiff; application=geotiff'}}}, 'B8': {'stac_asset': {'args': {'chunks': {}, 

In [ ]:
## Go up one level to `stac_cat['extensions-collection']['proj-example']` 
#print(l3.yaml()) # Contains the problematic typing `&id001 !!python/object:pystac.item.Item` throughout
#print(l3.serialize()) # Contains the problematic typing `&id001 !!python/object:pystac.item.Item` throughout

In [4]:
kwargs = l3._captured_init_kwargs.copy()
meta = kwargs.pop('metadata', l3.metadata) or {}
stac_obj = kwargs.pop('stac_obj') # Remove this arg that maps to the problematic typing and see if works
kwargs.update(dict(zip(inspect.signature(l3.__init__).parameters,
                l3._captured_init_args)))

l3data = {
    'driver': l3.classname,
    'description': l3.description,
    'metadata': l3.metadata,
    'args': kwargs
        }

l3dict = {
    'sources':
        {l3.name: l3data}
        }

with open('l3test.yaml', 'w') as f:
    f.write(yaml.dump(l3dict, sort_keys=False)) #Can be read in!

In [12]:
## Now include the sub catalogs b1 and b8

data = {'metadata':{'version':1}, 'sources':{}}
data['sources'][l3.name]=l3data

for key, source in l3.items():
    data['sources'][l3.name]['args']['stac_obj']['assets'][key] = source._yaml()['sources']

#print(yaml.dump(data, sort_keys=False))

with open('test.yaml', 'w') as f:
    f.write(yaml.dump(data, sort_keys=False))

# Full Circle: Load Intake Catalogue from .YAML

In [15]:
catalog2 = intake.open_catalog('test.yaml')
list(catalog2)

['proj-example']

In [ ]:
catalog2['proj-example'] #Type Error missing `stac_obj` - the key that I popped, so perhaps I did something taboo
#When I add this in like 
# sources:
#   proj-example:
#     stac_obj:
#       driver: intake_stac.catalog.StacItem
# I get a missing required key `driver` Validation Error